<a href="https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/T5/Fine_tuning_Dutch_T5_base_on_CNN_Daily_Mail_for_summarization_(on_TPU_using_HuggingFace_Accelerate).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# mindspore DEMO

using mindspore and mindnlp to develop a task fine-tuning T5

In [2]:
!pip install mindnlp

Looking in indexes: http://192.168.0.122:8888/repository/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 61.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.3/336.3 MB 13.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 62.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.1/450.1 kB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 66.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.8/316.8 kB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 49.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 18.4 MB/s eta 0:00:0

## Load dataset

Here we load the Dutch translation of the CNN/Daily Mail dataset, created by the Belgian AI company [ML6](https://www.ml6.eu/). It is hosted on HuggingFace's hub, as can be seen [here](https://huggingface.co/datasets/ml6team/cnn_dailymail_nl).

In [3]:
from mindnlp.dataset import load_dataset

#train_ds= load_dataset("", split=['train', 'test'])
ds = load_dataset("abisee/cnn_dailymail", "3.0.0", split=['train', 'validation','test'])

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/anaconda3/envs/MindSpore/lib

In [4]:
train = ds['train']
val =ds['validation']
test = ds['test']

In [5]:
import mindspore
import numpy as np
from mindspore.dataset import GeneratorDataset, transforms

Let's look at an example:

In [6]:
example = train[0]
example

('LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details 

In [7]:
article = example[0]
summary = example[1]
print("Article:", article)
print("Summary:", summary)

Article: LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Detai

In [8]:
from mindnlp.transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("flax-community/t5-base-dutch")

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/mindnlp/transformers/tokenization_utils_base.py:1526: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted, and will be then set to `False` by default. 
  warnings.warn(


In [37]:
prefix = "Summary: "
max_input_length = 512
from mindspore import Tensor
max_target_length = 64
class MyMapDataset():
    def __init__(self,data):
        super(MyMapDataset).__init__()
        self.data = data
    def __getitem__(self, index):
        
        articles,summaries,_ = self.data[index]
        #print(articles)
        inputs = [prefix + articles]
        model_inputs = tokenizer(inputs, max_length=max_input_length, padding="max_length", truncation=True)
        labels = [tokenizer(summaries, max_length=max_target_length, padding="max_length", truncation=True).input_ids]
        labels_with_ignore_index = []
       
        for labels_example in labels:
            labels_example = [label if label != 0 else -100 for label in labels_example]
            labels_with_ignore_index.append(labels_example)
        
        
        return model_inputs['input_ids'][0],model_inputs['attention_mask'][0],labels_with_ignore_index
    def __len__(self):
        return len(self.data)

In [38]:
train_data = MyMapDataset(train)

In [39]:
val_data = MyMapDataset(val)
test_data = MyMapDataset(test)

In [ ]:
train_data[0]

In [40]:
train_data =  mindspore.dataset.GeneratorDataset(train_data, column_names=["input_ids","attention_mask","labels"], shuffle=False)
#dataset = dataset.map(operations=transforms.TypeCast(mindspore.int32), input_columns="label", output_columns="labels")

Each example consists of an article and a corresponding summary. Easy, huh? Note that you can train T5 on any text-to-text problem. So it could be text as input as an SQL query (as text) as output, it could be text as input and a question related to that text as output (a task called question generation), etc.

## Encode the dataset

The T5 model, like any other Transformer model, does not directly expect text as input. Rather, it expects `input_ids` and `attention_mask`. The `input_ids` are integer vocabulary indices of the tokens of the text (you can read more about them [here](https://huggingface.co/transformers/glossary.html#input-ids)). As labels, it expects the `input_ids` of the summary. 

Let's encode them using the tokenizer. We also prepend the input with a so-called task prefix, which the authors of T5 used when fine-tuning the model. Here, the prefix is simply "Vat samen: " (which is Dutch for "Summarize: "), followed by a long document. Note that this prefix will probably not help a lot, since it has not seen this during pre-training. However, it is definitely useful if you want to fine-tune the English T5 model for summarization, as summarization is also part of its pre-training.

As we'll train the model on TPU, we pad both the inputs and targets up to the max length. If we were to train this model on GPUs, we would instead pad them up to the longest in a batch, which is more efficient in terms of memory. However, TPUs don't like that.

Let's verify an example, by decoding the `input_ids` back to text:

We also decode the `labels` (which aren't set to -100) back to text:

Next, let's set the format to PyTorch.

In [16]:
print("Number of training examples:", len(train_data))


Number of training examples: 287113


We define a function to create PyTorch dataloaders.

## Fine-tune a model 

Below, we define a `training_function`, which defines a regular training loop in native PyTorch. We only need to add a few lines to make sure the code will run on TPU. The Accelerator object will take care of that. Basically, the model as well as the data will be replicated across each of the 8 TPU cores. 

We also define a dictionary of training-related hyperparameters, which we can easily tweak.

In [17]:
hyperparameters = {
    "learning_rate": 0.0001,
    "num_epochs": 1, # set to very high number
    "seed": 42,
    "patience": 3, # early stopping
    "output_dir": "/content/",
}

In [18]:
from mindnlp.transformers import T5ForConditionalGeneration
model = T5ForConditionalGeneration.from_pretrained("flax-community/t5-base-dutch")

T5ForConditionalGeneration has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`.`PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB


In [ ]:
train_data[0][0]

In [20]:
a = model(input_ids= Tensor(train_data[0][0]),attention_mask = Tensor(train_data[0][1]),
         decoder_input_ids = Tensor(train_data[0][2]))

In [ ]:
model.trainable_params()

In [ ]:
params = []

In [24]:
import mindspore
from mindnlp.transformers import T5ForConditionalGeneration

from mindnlp.core import nn, ops, optim

from tqdm import tqdm
import mindnlp.transformers
from mindspore import value_and_grad
model = T5ForConditionalGeneration.from_pretrained("flax-community/t5-base-dutch")

def forward_fn(data, labels):
    input_ids = Tensor(data[0])
    attention_mask = Tensor(data[1])
    labels = Tensor(labels)
    outputs = model(input_ids= input_ids,attention_mask =attention_mask,labels = labels)
    loss = outputs.loss
    #print(loss)
    return loss


def training_function(train_dataset,test_dataset):
   

    # Instantiate optimizer
    optimizer = optim.AdamW(
                        model.trainable_params(),
        lr=hyperparameters["learning_rate"])

  
    
    # Now we train the model
    epochs_no_improve = 0
    min_val_loss = 1000000
    weights = ()
    for group in optimizer.param_groups:
            weights += tuple(group['params'])
            
    grad_fn = value_and_grad(forward_fn,grad_position=1)
    running_loss = 0.
    for epoch in range(hyperparameters["num_epochs"]):
        
        model.train()
        running_loss = 0.
        for i in tqdm(range(200)):
            data,label = (train_data[i][0],train_data[i][1]),train_data[i][2]
            
            
            #loss = forward_fn(data,label)
            loss, grads = grad_fn(data, label)
            running_loss += loss.asnumpy()
            
            
            optimizer.step(grads)


Next, we can easily start training by wrapping the `training_function` in a `notebook_launcher`.

In [25]:
training_function(train_data,test_data)

100%|██████████| 200/200 [18:14<00:00,  5.47s/it]


## Inference

Now that we have a trained model, let's use it to generate a summary on a new, unseen text.

In [30]:
text = """Artificial Intelligence (AI) has rapidly evolved and become an integral part of various industries, 
particularly healthcare. AI's ability to analyze large volumes of data and identify patterns that humans might miss 
has revolutionized many aspects of medical care, 
from diagnostics to treatment planning and patient monitoring.
One of the most significant contributions of AI in healthcare is in the field of diagnostics. 
Machine learning algorithms, a subset of AI, have been trained to interpret medical imaging data, such as X-rays, MRIs, and CT scans.
These algorithms can often detect abnormalities, such as tumors or fractures, with higher accuracy and speed than human radiologists. 
This not only improves the speed of diagnosis but also reduces the potential for human error, leading to better patient outcomes.
AI has also shown promise in personalized medicine, where treatments are tailored to an individual's genetic makeup and health history.
By analyzing patient data, including genetic information, 
AI systems can recommend specific treatment plans that are more likely to be effective. 
This can greatly enhance the precision of treatments, reduce side effects, and improve overall patient care.."""

#trained_model = T5ForConditionalGeneration.from_pretrained(hyperparameters["output_dir"])

input_ids = tokenizer(text, max_length=max_input_length, padding="max_length", truncation=True).input_ids
input_ids = Tensor([input_ids])
generated_ids = model.generate(input_ids, do_sample=True, 
    max_length=50, 
    top_k=0, 
    temperature=0.7
)

summary = tokenizer.decode(generated_ids.squeeze(), skip_special_tokens=True)
print(summary)

editing. AI verkent acute science and medical healthcare applications, such as big data, and receives tests to treatments..


Make sure to upload the trained model to the hub, such that other people can re-use it.